In [1]:

import pandas as pd

## Loading all the CSV Files collected from GitHubAPI

In [2]:
all_csv_files = []

In [3]:
files= ["parallel_Exec_data_2_0.1_samples.csv",
"parallel_Exec_data.csv",
"parallel_Exec_datafile_2023-11-19_02-18-04.csv",
"parallel_Exec_datafile_2023-11-19_02-04-02.csv",
"parallel_Exec_datafile_2023-11-19_01-39-50.csv"]

In [4]:
for file in files:
    df = pd.read_csv(file)
    all_csv_files.append(df)

In [5]:
df = pd.concat(all_csv_files)

In [6]:
df.shape

(101853, 22)

### Removing the duplicate Records


In [7]:
df2 = df.drop_duplicates()

In [8]:
df2.shape

(101837, 22)

In [9]:
file_path = r"preprocessed_github_user_repository_data.csv"


In [10]:
df3 = pd.read_csv(file_path)

### Integrating the users repository data

In [11]:
final_df = pd.concat([df2, df3])

In [12]:
final_df.shape

(443529, 23)

In [13]:
final_df.drop(columns=['visibility', 'contributors_url', 'languages_url', 'description', 'default_branch','full_repository_name'], inplace=True)

In [14]:
final_df.drop(columns=['user_name'] , inplace=True)

## Handling column names

In [15]:
final_df.columns = [col.replace(" ", "_") for col in final_df.columns]

In [16]:
final_df.drop(columns=['watchers', 'open_issues'],inplace=True)

In [17]:
final_df.drop_duplicates(inplace=True)

In [18]:
final_df.shape

(443529, 14)

In [19]:
numeric_columns = final_df.select_dtypes(include=['number']).columns

date_columns = ['created_at', 'pushed_at','updated_at' ]


In [20]:
for col in date_columns:
    final_df[col] = pd.to_datetime(final_df[col], format='mixed', dayfirst=True) 
        

In [21]:
categorical_columns = [col for col in df.columns if (col  not in numeric_columns) and (col not in date_columns)]


In [22]:
[df[col].fillna(0, inplace=True) for col in final_df.columns if col in numeric_columns]
       

[None, None, None, None]

In [23]:
for col in date_columns:
            final_df[col].fillna(method='bfill', inplace=True)

C:\Users\venka\AppData\Local\Temp\ipykernel_3060\957605254.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  final_df[col].fillna(method='bfill', inplace=True)


In [24]:
final_df['language_count'].fillna(0, inplace=True)

In [25]:
final_df.drop(final_df[final_df['language'].isnull()].index, inplace=True)

In [26]:
final_df.drop(columns=['visibility', 'contributors_url', 'languages_url', 'description', 'name', 'default_branch','full_repository_name'], inplace=True)


KeyError: "['visibility', 'contributors_url', 'languages_url', 'description', 'default_branch', 'full_repository_name'] not found in axis"

In [27]:
final_df['user_name'].fillna("not_given" , inplace=True)

In [28]:
final_df['name'].fillna("main", inplace=True)

In [29]:
final_df.isnull().sum()

user_name              0
state                  0
contributions_count    0
name                   0
user_type              0
created_at             0
pushed_at              0
updated_at             0
open_issues_count      0
watchers_count         0
is_disabled            0
is_archieved           0
language               0
language_count         0
dtype: int64

In [30]:
final_df.describe()

,contributions_count,open_issues_count,watchers_count,language_count
count,423035.000000,423035.000000,423035.000000,4.230350e+05
mean,369.196461,0.750619,11.577236,1.100926e+05
std,2186.650374,8.253676,320.406287,4.377601e+06
min,0.000000,0.000000,0.000000,0.000000e+00
25%,4.000000,0.000000,0.000000,0.000000e+00
50%,15.000000,0.000000,0.000000,0.000000e+00
75%,87.000000,0.000000,0.000000,0.000000e+00
max,111090.000000,596.000000,31810.000000,1.172791e+09


In [31]:
final_df['user_type'].value_counts()

user_type
User            405890
Organization     17145
Name: count, dtype: int64

In [32]:
from sklearn.utils import resample

In [33]:

users_data = final_df[final_df['user_type'] == 'User']
orgs_data = final_df[final_df['user_type'] == 'Organization']


sampled_data = resample(users_data, replace=True, n_samples=len(orgs_data))

balanced_df = pd.concat([sampled_data, users_data])


In [34]:
final_df.to_csv("balanced_dataset_dec_19.csv", index=False)

In [ ]:
final_df.describe